In [4]:
import gym 

env = gym.make("FrozenLake-v0")
env.render()


SFFF
FHFH
FFFH
HFFG


In [9]:
#Start→Goal +1  Holeにおちると-1
env.desc

array([[b'S', b'F', b'F', b'F'],
       [b'F', b'H', b'F', b'H'],
       [b'F', b'F', b'F', b'H'],
       [b'H', b'F', b'F', b'G']], dtype='|S1')

In [10]:
env.desc[3][2]

b'F'

In [14]:
[env.ncol,env.nrow] #何行何列か

[4, 4]

In [12]:
env.nS #状態（どのマスにいるか）の数

16

In [15]:
env.nA #行動（どの方向に進みうるか）の数

4

In [16]:
env.reset()
env.step(3) #0:Left, 1:Down, 2:Right, 3:Up
env.render()

  (Up)
SFFF
FHFH
FFFH
HFFG


In [17]:
env.P #{s:{a:(確率,遷移後の状態,,,)}

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [38]:
import pandas as pd

class Planner:
    def __init__(self, env):
        self.env = env
        
    def s_to_loc(self, s):
        row = s// self.env.ncol
        col = s% self.env.ncol
        return row, col
    
    def reward(self, s):
        row, col = self.s_to_loc(s)
        if self.env.desc[row][col] == b'G':
            return 1
        elif self.env.desc[row][col] == b'H':
            return -1
        else:
            return 0
        
    def plan(self, gamma=0.9, threshold=0.0001):
        self.env.reset()
        V = {}
        for s in range(self.env.nS):
            V[s] = self.reward(s)
        while True:
            delta = 0
            for s in V:
                row, col = self.s_to_loc(s)
                if self.env.desc[row][col] in [b'G', b'H']:
                    continue #GとHからは行動できないため処理を飛ばす
                expected_rewards = []
                for a in range(self.env.nA):
                    r = 0
                    for prob, next_s, _, _ in self.env.P[s][a]: # 「_」は取り出さないとき
                        r += gamma * prob * V[next_s]
                    expected_rewards.append(r)
                new_V = self.reward(s) + max(expected_rewards)

                delta = max(delta, abs(new_V - V[s]))
                V[s] = new_V

            if delta < threshold:
                break
                
        return self.dict_to_grid(V)
    
    def dict_to_grid(self, V):
        grid = []
        for i in range(self.env.nrow):
            row = [0]*self.env.ncol
            grid.append(row)
        for s in V:
            row, col = self.s_to_loc(s)
            grid[row][col] = V[s]
        return pd.DataFrame(grid)

In [39]:
pl = Planner(env)
pl.env.reset()
pl.env.render()


SFFF
FHFH
FFFH
HFFG


In [40]:
pl.reward(1)

0

In [41]:
pl.plan()

,0,1,2,3
0,0.045684,0.026631,0.016628,0.01234
1,0.061139,-1.000000,-0.252228,-1.00000
2,0.097082,0.165458,0.142632,-1.00000
3,-1.000000,0.311850,0.562218,1.00000


In [ ]:
# 動的計画表
# 状態sに対して、価値V(s)を計算し、価値が最大になる行動aをとる
# V(s) = Reward(s)+減衰率*次の行動の期待値
# －価値反復法
# 最初、V(s)を適当に決める